In [21]:
import sys  
sys.path.insert(1, './../../../ESN-IP/IP/.')

import random

random.seed(10)

from DATA import MG17

data = MG17(True, [80,0,20])

X_TR, Y_TR = data.TR()
X_VAL, Y_VAL = data.VAL()
X_TS, Y_TS = data.TS()


In [24]:
from Metrics import MSE
from IPReservoir import IPReservoir
from ESN import Reservoir, EchoStateNetwork
from IntrinsicPlasticity import IPMask, IPDistribution

N_UNITS_LIST = [100]
W_range=(-0.5, 0.5)
initial_rho = 0.5
desired_rhos = [0.95]
bias_range = [-0.1, 0.1]


In [27]:
REPETITIONS = 5
max_epochs = 25 
columns = ["Name", "TYPE", "MSE", "MC", "Lambda max", "delta_phI", "KL", "Rho", "A-Norm", "B-Norm"]
models_num = len(N_UNITS_LIST)*len(desired_rhos)
rows = []

for N_UNITS, desired_rho in zip(N_UNITS_LIST, desired_rhos):
    config_string = f"{N_UNITS} UNIT - Spectral Radius: {desired_rho}"
    names = ["Vanilla", "Gauss 1", "Gauss 2", "Bimodal", "Trim 1", "Trim 2"]
    identifiers = [ f"- {name} - {config_string} " for name in names]
    
    for R in range(REPETITIONS):
        vanilla_reservoir = Reservoir(N = N_UNITS, ro_rescale = desired_rho, W_range=W_range, bias_range=bias_range)
        
        gaussian_reservoir_1 = IPReservoir(N = N_UNITS, ro_rescale=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.gaussian(N_UNITS, 0.5, apply_activation=True))
        gaussian_reservoir_2 = IPReservoir(N = N_UNITS, ro_rescale=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.gaussian(N_UNITS, 0.25, apply_activation=True))

        bimodal_reservoir = IPReservoir(N = N_UNITS, ro_rescale=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.mixedBimodalMask(N_UNITS, 0.69, 0.92, apply_activation=True))
        
        trimodal_reservoir_1 = IPReservoir(N = N_UNITS, ro_rescale=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.trimodal(N_UNITS, 9/10, 0.25, 0.15, 0.92, apply_activation=True))
        trimodal_reservoir_2 = IPReservoir(N = N_UNITS, ro_rescale=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.trimodal(N_UNITS, 9/10, 0.25, 0.075, 0.72, apply_activation=False))
        
        print("Traning IP models")
        gaussian_reservoir_1.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, eta=0.000005, verbose=False)
        gaussian_reservoir_2.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, eta=0.000005, verbose=False)
        bimodal_reservoir.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, eta=0.000005,verbose=False)
        trimodal_reservoir_1.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, eta=0.000005, verbose=False)
        trimodal_reservoir_2.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, eta=0.000001, verbose=False)

        models = [vanilla_reservoir, gaussian_reservoir_1, gaussian_reservoir_2, bimodal_reservoir, trimodal_reservoir_1, trimodal_reservoir_2]

        for model_index in range(len(models)): 
            model = models[model_index]

            esn = EchoStateNetwork(model)
            esn.train(X_TR,Y_TR, 0)

            mse = esn.evaluate(X_TS, Y_TS, metric=MSE()).item()
            
            mc = esn.MemoryCapacity().item()
            
            lambda_max = model.LCE(X_TS)

            de_phi = model.de_fi()

            KL = model.kl_value.item() if model_index != 0 else None
            a_norm = model.a.norm().item() if model_index != 0 else 1
            b_norm = model.b.norm().item() if model_index != 0 else 0

            # columns = ["________________Name____________________","TYPE", "MSE","MC","Lambda max","delta_phI","KL", "Rho", "A-Norm", "B-Norm"]
            rows.append([f"{R+1}) {identifiers[model_index]}" , model_index, mse, mc, lambda_max, de_phi, KL, model.max_eigs().item(), a_norm, b_norm])
            print(rows[-1])

        
        

Traning IP models
Reservoir warmed up with the first 100 time steps
['1) - Vanilla - 100 UNIT - Spectral Radius: 0.95 ', 0, 6.201832881933513e-09, 17.257631301879883, -0.11910155, 0.06256304670592634, None, 0.9499996900558472, 1, 0]
Reservoir warmed up with the first 100 time steps


C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=6.74887e-10): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=6.57286e-10): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=6.63514e-10): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Lo

['1) - Gauss 1 - 100 UNIT - Spectral Radius: 0.95 ', 1, 1.6001564709820197e-08, 9.348576545715332, -0.26051438, 0.09167585022909536, 0.228987455368042, 0.9489169120788574, 16.26150894165039, 3.844196319580078]
Reservoir warmed up with the first 100 time steps


C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.09359e-12): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.75649e-12): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


['1) - Gauss 2 - 100 UNIT - Spectral Radius: 0.95 ', 2, 2.718573679626395e-08, 9.87408447265625, -0.12505093, 0.049234474704391254, 0.15802685916423798, 0.9492663145065308, 13.565071105957031, 4.2238874435424805]
Reservoir warmed up with the first 100 time steps


...

['5) - Gauss 1 - 100 UNIT - Spectral Radius: 0.95 ', 1, 2.0705786985829445e-08, 9.80918025970459, -0.9165455, 0.04392482873496852, 0.22113174200057983, 0.9486987590789795, 18.45478630065918, 4.870870113372803]
Reservoir warmed up with the first 100 time steps
['5) - Gauss 2 - 100 UNIT - Spectral Radius: 0.95 ', 2, 3.573847964001357e-08, 8.90561580657959, -0.26105845, 0.0619435132260412, 0.18563227355480194, 0.9498488903045654, 14.755616188049316, 5.055260181427002]
Reservoir warmed up with the first 100 time steps


C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=6.10555e-10): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=6.09835e-10): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=6.07239e-10): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Lo

['5) - Bimodal - 100 UNIT - Spectral Radius: 0.95 ', 3, 5.3940375783468097e-08, 7.9559502601623535, -0.79590285, 0.15500821302114048, 0.1070946529507637, 0.9495863914489746, 18.67742156982422, 9.421621322631836]
Reservoir warmed up with the first 100 time steps


C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.16389e-11): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.8208e-11): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.90753e-12): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\casa_\AppData\Loc

['5) - Trim 1 - 100 UNIT - Spectral Radius: 0.95 ', 4, 4.361556343467503e-08, 9.069546699523926, -0.16576445, 0.044169367666638495, 0.08461140096187592, 0.9495558142662048, 20.73088264465332, 9.14965534210205]
Reservoir warmed up with the first 100 time steps
['5) - Trim 2 - 100 UNIT - Spectral Radius: 0.95 ', 5, 3.710356358706818e-08, 9.159811973571777, -0.090210535, 0.5312170724735811, 0.050157591700553894, 0.949726939201355, 20.3798885345459, 9.066017150878906]


In [28]:
import pandas as pd 
df = pd.DataFrame(rows)
df.to_csv("./RESULTS_neg_MG17.csv", header=columns)

In [29]:
df = pd.DataFrame(rows, columns=columns)

In [30]:
df

,Name,TYPE,MSE,MC,Lambda max,delta_phI,KL,Rho,A-Norm,B-Norm
0,1) - Vanilla - 100 UNIT - Spectral Radius: 0.95,0,6.201833e-09,17.257631,-0.119102,0.062563,NaN,0.950000,1.000000,0.000000
1,1) - Gauss 1 - 100 UNIT - Spectral Radius: 0.95,1,1.600156e-08,9.348577,-0.260514,0.091676,0.228987,0.948917,16.261509,3.844196
2,1) - Gauss 2 - 100 UNIT - Spectral Radius: 0.95,2,2.718574e-08,9.874084,-0.125051,0.049234,0.158027,0.949266,13.565071,4.223887
3,1) - Bimodal - 100 UNIT - Spectral Radius: 0.95,3,5.130924e-08,7.874024,-3.442305,0.343772,0.105865,0.949620,18.847832,9.154375
4,1) - Trim 1 - 100 UNIT - Spectral Radius: 0.95,4,2.219853e-08,9.958858,-0.120730,0.104644,0.066367,0.949827,17.470385,7.730860
5,1) - Trim 2 - 100 UNIT - Spectral Radius: 0.95,5,2.793126e-08,9.538047,-0.062013,0.050965,0.051632,0.949950,18.866203,8.413270
6,2) - Vanilla - 100 UNIT - Spectral Radius: 0.95,0,8.051179e-09,18.305702,-0.059742,0.077035,NaN,0.950000,1.000000,0.000000
7,2) - Gauss 1 - 100 UNIT - Spectral Radius: 0.95,1,2.505252e-08,9.449042,-0.268086,0.170868,0.241310,0.949452,17.258587,4.494400
8,2) - Gauss 2 - 100 UNIT - Spectral Radius: 0.95,2,3.004722e-08,8.828341,-1.066588,0.153403,0.219468,0.949618,18.219397,7.017938
9,2) - Bimodal - 100 UNIT - Spectral Radius: 0.95,3,4.273440e-08,8.202055,-3.081004,0.044233,0.108411,0.949881,18.487213,8.777704


In [32]:
df.groupby("TYPE").mean()

,MSE,MC,Lambda max,delta_phI,KL,Rho,A-Norm,B-Norm
TYPE,,,,,,,,
0,6.721856e-09,18.172340,-0.089154,0.067212,NaN,0.950000,1.000000,0.000000
1,1.904865e-08,9.444393,-1.140481,0.086087,0.237048,0.947838,17.258138,4.410811
2,3.393488e-08,8.774029,-1.002262,0.142185,0.204661,0.949554,17.048490,6.412317
3,4.781955e-08,8.247719,-1.674048,0.184353,0.104102,0.949620,18.952129,9.459829
4,3.195886e-08,9.451917,-0.150578,0.093909,0.081512,0.949743,18.885298,8.555701
5,2.623987e-08,9.573828,-0.084395,0.247099,0.052355,0.949800,19.155162,8.708235
